In [1]:
path = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/covid/10k_synthea_covid19_csv/"
observations_file = path + "observations.csv"
conditions_file = path +  "conditions.csv"
care_plans_file = path + "careplans.csv"
patients_file = path + "patients.csv"
encounters_file = path + "encounters.csv"
# devices_file = path + "devices.csv"
# supplies_file = path + "supplies.csv"
# procedures_file = path + "procedures.csv"
# medications_file = path + "medications.csv"

observations = spark.read.format("csv").option("inferSchema", True) \
.option("header", True).load(observations_file)

conditions = spark.read.format("csv").option("inferSchema", True) \
.option("header", True).load(conditions_file)

care_plans = spark.read.format("csv").option("inferSchema", True) \
.option("header", True).load(care_plans_file)

patients = spark.read.format("csv").option("inferSchema", True) \
.option("header", True).load(patients_file)

encounters = spark.read.format("csv").option("inferSchema", True) \
.option("header", True).load(encounters_file)

# devices = spark.read.format("csv").option("inferSchema", True) \
# .option("header", True).load(devices_file)

# supplies = spark.read.format("csv").option("inferSchema", True) \
# .option("header", True).load(supplies_file)

# procedures = spark.read.format("csv").option("inferSchema", True) \
# .option("header", True).load(procedures_file)

# medications = spark.read.format("csv").option("inferSchema", True) \
# .option("header", True).load(medications_file)



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1597842492404_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Format a dataframe of patients who have covid-19

conditions.createOrReplaceTempView("conditions")
patients.createOrReplaceTempView("patients")

covid_ids = spark.sql("""
select PATIENT from conditions 
where CODE = 840539006""")

covid_ids.createOrReplaceTempView("ids")

covid_patients = spark.sql("""
select p.* from patients p join ids i
on p.Id = i.PATIENT""")

covid_patients.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8820

In [3]:
# create a deceased column

from pyspark.sql import functions 
covid_patients.createOrReplaceTempView("covid_patients")

covid_patients = covid_patients.withColumn("DECEASED", 
                                           functions.when(covid_patients.DEATHDATE.isNotNull(), 1)
                                          .otherwise(0))

# covid_patients.select("DEATHDATE", "DECEASED").where(covid_patients.DEATHDATE.isNotNull()).show()
covid_patients.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8820

In [11]:
data = covid_patients

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# observations.createOrReplaceTempView("observations")
# aliases = ["Height", "Weight", "BMI", "Cholesterol", "Sodium", "Glucose"]
# columns = ["Body Height", "Body Weight", "Body Mass Index", "Total Cholesterol", "Sodium", "Glucose"]

# queries = []
# for column, alias in zip(columns, aliases):
#     queries.append(f"""
#     select PATIENT, last(VALUE) as {alias} 
#     from observations
#     where DESCRIPTION = "{column}" 
#     group by PATIENT
#     """)


# for indx, query in enumerate(queries):
#     data.createOrReplaceTempView("data")
    
#     temp_table = spark.sql(query)
#     temp_table.createOrReplaceTempView("temp_table")
    

#     data = spark.sql(f"""
#     select d.*, t.{aliases[indx]}
#     from data d 
#     join temp_table t
#     on d.Id = t.PATIENT""")
    
# # data.show(1, truncate = True)

# data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1831

In [8]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# selecting relevant observations for each patient

observations.createOrReplaceTempView("observations")
aliases = ["BMI", "Cholesterol"]
columns = ["Body Mass Index", "Total Cholesterol"]

queries = []
for column, alias in zip(columns, aliases):
    queries.append(f"""
    select PATIENT, last(VALUE) as {alias} 
    from observations
    where DESCRIPTION = "{column}" 
    group by PATIENT
    """)


for indx, query in enumerate(queries):
    data.createOrReplaceTempView("data")
    
    temp_table = spark.sql(query)
    temp_table.createOrReplaceTempView("temp_table")
    

    data = spark.sql(f"""
    select d.*, t.{aliases[indx]}
    from data d 
    left outer join temp_table t
    on d.Id = t.PATIENT""")
    
# data.show(1, truncate = True)

data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8820

In [27]:
data = data.select("Id", "BIRTHDATE", "DEATHDATE", "MARITAL", "RACE", "ETHNICITY", "GENDER", \
                  "HEALTHCARE_EXPENSES", "HEALTHCARE_COVERAGE", "DECEASED", "BMI", "Cholesterol")

# data.select("HEALTHCARE_EXPENSES", "HEALTHCARE_COVERAGE").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# add average lab observations from the last day in the lab for each patient

observations.createOrReplaceTempView("observations")
aliases = ["D_dimer", "Serum_Ferritin", "High_Sensitivity_Cardiac_Troponin_I", \
           "IL_6", "Lymphocytes", "Lactate_dehydrogenase"]

columns = ["48065-7", "2276-4", "89579-7", \
           "26881-3", "731-0", "14804-9"]

queries = []
for column, alias in zip(columns, aliases):
    queries.append(f"""
    select PATIENT, avg(VALUE) as {alias} 
    from observations
    where CODE = "{column}" 
    group by PATIENT
    """)


for indx, query in enumerate(queries):
    data.createOrReplaceTempView("data")
    
    temp_table = spark.sql(query)
    temp_table.createOrReplaceTempView("temp_table")
    

    data = spark.sql(f"""
    select d.*, t.{aliases[indx]}
    from data d 
    left join temp_table t
    on d.Id = t.PATIENT""")
    
# data.select(aliases).show(5, truncate = True)
# data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
data.createOrReplaceTempView("data")

for alias in aliases:
    spark.sql(f"""
    select count(*) from data
    where {alias} is null""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|    6953|
+--------+

+--------+
|count(1)|
+--------+
|    6953|
+--------+

+--------+
|count(1)|
+--------+
|    6953|
+--------+

+--------+
|count(1)|
+--------+
|    8802|
+--------+

+--------+
|count(1)|
+--------+
|    6953|
+--------+

+--------+
|count(1)|
+--------+
|    6953|
+--------+

In [24]:
# Add icu_admit column
data.createOrReplaceTempView("data")
encounters.createOrReplaceTempView("encounters")

# from pyspark.sql import functions

# data.withColumn("icu_admit", functions.when((data.Id == encounters.PATIENT & encounters.CODE == 305351004), 1.0)
#                .otherwise(0.0))

icu_encounters = spark.sql("""
select  PATIENT, DESCRIPTION
from encounters
where CODE = 305351004""")


# data.select("icu_admit").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
icu_encounters.show(1, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------+--------------------------------------------+
|PATIENT                             |DESCRIPTION                                 |
+------------------------------------+--------------------------------------------+
|c70992c9-ff13-467b-9032-1901506edeef|Admission to intensive care unit (procedure)|
+------------------------------------+--------------------------------------------+
only showing top 1 row

In [29]:
icu_encounters.createOrReplaceTempView("icu_encounters")
data.createOrReplaceTempView("data")

data = spark.sql("""
select d.*, i.DESCRIPTION as icu_admitted from 
data d left outer join icu_encounters i
on d.Id = i.PATIENT
""")

from pyspark.sql.functions import translate

data = data.withColumn('icu_admitted', translate('icu_admitted', 
                                                        'Admission to intensive care unit (procedure)', '1'))

data = data.na.fill(value='0', subset='icu_admitted')

data.createOrReplaceTempView("data")

spark.sql("""
select count(*) from data where 
icu_admitted is not null""").show()
# data.select("icu_admitted").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|    8820|
+--------+

In [32]:
data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'BIRTHDATE', 'DEATHDATE', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'DECEASED', 'BMI', 'Cholesterol', 'D_dimer', 'Serum_Ferritin', 'High_Sensitivity_Cardiac_Troponin_I', 'IL_6', 'Lymphocytes', 'Lactate_dehydrogenase', 'icu_admitted']

In [39]:
# selecting relevent columns

# covid_data = data.select("Id", "BIRTHDATE", "MARITAL", "RACE", "ETHNICITY", "GENDER", "CITY", 'STATE', 'COUNTY', \
#                         'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'DECEASED', 'Height', 'Weight', 'BMI', 'Cholesterol', \
#                          'Sodium', 'Glucose', 'D_dimer', 'Serum_Ferritin', 'High_Sensitivity_Cardiac_Troponin_I', 'IL_6', 'Lymphocytes', 'Lactate_dehydrogenase', 'icu_admitted')
covid_data = data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Indexing categorical features

In [40]:
from pyspark.ml.feature import StringIndexer

input_cols = ["MARITAL", "RACE", "ETHNICITY", "GENDER",'DECEASED', 'icu_admitted']


from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(covid_data) \
            for column in input_cols]

pipeline = Pipeline(stages=indexers)
indexed = pipeline.fit(covid_data).transform(covid_data)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:

    
indexed = indexed.select(*['Id', 'BIRTHDATE','BMI', 'Cholesterol', \
                          'D_dimer', 'Serum_Ferritin', \
                          'High_Sensitivity_Cardiac_Troponin_I', 'IL_6', \
                           'Lymphocytes', 'Lactate_dehydrogenase', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'], \
                          *[column + "_index" for column in input_cols])

indexed.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'BIRTHDATE', 'BMI', 'Cholesterol', 'D_dimer', 'Serum_Ferritin', 'High_Sensitivity_Cardiac_Troponin_I', 'IL_6', 'Lymphocytes', 'Lactate_dehydrogenase', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'MARITAL_index', 'RACE_index', 'ETHNICITY_index', 'GENDER_index', 'DECEASED_index', 'icu_admitted_index']

In [ ]:
# indexed_n = indexed.select(['Id', 'BIRTHDATE', 'Height', 'Weight', 'BMI', \
#                             'Cholesterol', 'Sodium', 'Glucose', 'D_dimer', 'Serum_Ferritin', \
#                             'High_Sensitivity_Cardiac_Troponin_I', \
#                             'IL_6', 'Lymphocytes', 'Lactate_dehydrogenase', 'MARITAL_index', 'RACE_index', \
#                             'ETHNICITY_index', 'GENDER_index', 'CITY_index', 'STATE_index', \
#                             'COUNTY_index', 'HEALTHCARE_EXPENSES_index', 'HEALTHCARE_COVERAGE_index', \
#                             'DECEASED_index', 'icu_admitted_index'])

In [43]:
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

today = datetime.today()

convert_to_age = udf(lambda birth_date: today.year - birth_date.year, IntegerType())

indexed = indexed.withColumn("AGE", convert_to_age(indexed.BIRTHDATE))

indexed = indexed.drop("BIRTHDATE")

# indexed.select("AGE").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Convert cells to Floats

In [47]:
cols_to_convert = ['BMI', 'Cholesterol', \
                          'D_dimer', 'Serum_Ferritin', \
                          'High_Sensitivity_Cardiac_Troponin_I', 'IL_6', \
                            'Lymphocytes', 'Lactate_dehydrogenase', 'AGE']

from pyspark.sql.types import FloatType

for col in cols_to_convert:
    indexed = indexed.withColumn(col, indexed[col].cast(FloatType()))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Impute Null Values

In [48]:

from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=cols_to_convert, outputCols=cols_to_convert)
indexed = imputer.fit(indexed).transform(indexed)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
cols = indexed.columns
cols.remove("MARITAL_index")
indexed_n = indexed.select(cols)
indexed_n.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'BMI', 'Cholesterol', 'D_dimer', 'Serum_Ferritin', 'High_Sensitivity_Cardiac_Troponin_I', 'IL_6', 'Lymphocytes', 'Lactate_dehydrogenase', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'RACE_index', 'ETHNICITY_index', 'GENDER_index', 'DECEASED_index', 'icu_admitted_index', 'AGE']

In [62]:
indexed_n.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----------+---------+--------------+-----------------------------------+----+-----------+---------------------+-------------------+-------------------+----------+---------------+------------+--------------+------------------+----+
|                  Id|      BMI|Cholesterol|  D_dimer|Serum_Ferritin|High_Sensitivity_Cardiac_Troponin_I|IL_6|Lymphocytes|Lactate_dehydrogenase|HEALTHCARE_EXPENSES|HEALTHCARE_COVERAGE|RACE_index|ETHNICITY_index|GENDER_index|DECEASED_index|icu_admitted_index| AGE|
+--------------------+---------+-----------+---------+--------------+-----------------------------------+----+-----------+---------------------+-------------------+-------------------+----------+---------------+------------+--------------+------------------+----+
|00ebc091-9748-42c...|25.989397|  183.77635|      0.0|         498.0|                                3.0| 6.0|        0.0|                244.0|             6236.4|                0.0|       1.0|            0

In [64]:
# Save the indexed dataframe here for future use:

# indexed_n.coalesce(1).write.csv("s3://ms-syntheamass-1m-mldata/syntheamassCSV/covid/10k_synthea_covid19_csv/", 
#                              header = 'true', mode="append")

# you can resume from here using the indexed dataframe from the s3 bucket

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# assemble a features column

cols = indexed_n.columns
cols.remove("DECEASED_index")
cols.remove("Id")

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols, outputCol='features')

indexed_n = assembler.transform(indexed_n)
# indexed.select("features").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Scale the features column

In [22]:
from pyspark.ml.feature import StandardScaler
standardscaler = StandardScaler().setInputCol("features").setOutputCol("scaled_features")

scaled = standardscaler.fit(indexed).transform(indexed)
# scaled.select("scaled_features").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Train, Test, split

In [25]:
train, test = scaled.randomSplit([0.8, 0.2], seed = 10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
percent_pos_total = float(scaled.select("DECEASED_index").where("DECEASED_index == 1").count()) \
/ scaled.count() * 100

print("percent deceased for total covid patients:", percent_pos_total)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

percent deceased for total covid patients: 16.666666666666664

In [26]:
percent_pos_train = float(train.select("DECEASED_index").where("DECEASED_index == 1").count()) \
/ train.count() * 100

percent_pos_test = float(test.select("DECEASED_index").where("DECEASED_index == 1").count()) \
/ test.count() * 100

print("percent deceased for train:", percent_pos_train)

print("percent deceased for test:", percent_pos_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

percent deceased for train: 25.0
percent deceased for test: 0.0

# Imbalance handling


In [27]:
num_negs = float(train.where("DECEASED_index == 0").count())
                 
balancing_ratio = num_negs/train.count()

# print("Balancing ratio:", balancing_ratio)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
from pyspark.sql import functions

train = train.withColumn("class_weights", functions.when(train.DECEASED_index == 1, balancing_ratio)
                        .otherwise(1 - balancing_ratio))
# train.select("class_weights").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Select Features

In [29]:
# Feature selection using chisquareSelector
# selects significant features 

from pyspark.ml.feature import ChiSqSelector

css = ChiSqSelector(featuresCol='scaled_features', outputCol = 'Aspect', 
                   labelCol='DECEASED_index', fpr=0.05)

train = css.fit(train).transform(train)

test = css.fit(test).transform(test)

# test.select("Aspect").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Building a classification Model using Logistic Regression

In [30]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="DECEASED_index", 
                       featuresCol="Aspect", weightCol = "class_weights", maxIter = 10)
model = lr.fit(train)
predict_train = model.transform(train)

predict_test = model.transform(test)
# predict_test.select("DECEASED_index", "prediction").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Model Evaluation

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator 
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="DECEASED_index")

predict_test.select("DECEASED_index", "rawPrediction", "prediction", "probability").show(5)

print("The area under ROC for train set is:", evaluator.evaluate(predict_train))
print("The area under ROC for test set is:", evaluator.evaluate(predict_test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://172.31.21.238:18888/sessions/0/statements/32 with error payload: {"msg":"requirement failed: Session isn't active."}


In [33]:
sc.install_pypi_package("matplotlib")

VBox()

An error was encountered:
Session 0 unexpectedly reached final status 'dead'. See logs:
stdout: 
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (mmap) failed to map 101183488 bytes for committing reserved memory.
# An error report file with more information is saved as:
# /tmp/hs_err_pid27258.log

stderr: 
20/08/18 19:23:44 INFO DAGScheduler: Submitting ShuffleMapStage 1283 (MapPartitionsRDD[4467] at rdd at BinaryClassificationEvaluator.scala:81), which has no missing parents
20/08/18 19:23:44 INFO MemoryStore: Block broadcast_1154 stored as values in memory (estimated size 20.9 KB, free 837.2 MB)
20/08/18 19:23:44 INFO MemoryStore: Block broadcast_1154_piece0 stored as bytes in memory (estimated size 10.4 KB, free 837.2 MB)
20/08/18 19:23:44 INFO BlockManagerInfo: Added broadcast_1154_piece0 in memory on ip-172-31-21-238.ec2.internal:39767 (size: 10.4 KB, free: 1003.4 MB)
20/08/18 19:23:44 INFO SparkContext: Created broadcast 

In [29]:
import matplotlib.pyplot as plt
import numpy as np

beta = np.sort(model.coefficients)

plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()



VBox()

An error was encountered:
Invalid status code '404' from https://172.31.21.238:18888/sessions/3 with error payload: {"msg":"Session '3' not found."}


In [ ]:
from pyspark.ml.classification import LogisticRegression

trainingSummary = model.summary

roc = trainingSummary.roc.toPandas()


accuracy = trainingSummary.accuracy

print(accuracy)
